# Group 19: Milestone 1

<!--  -->

## Library Imports

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import dask.dataframe as dd

<!--  -->

## 3. Downloading the data

rubric={correctness:10}

<div class="alert alert-block alert-info">
Download the data from figshare to your local computer using the figshare API (you need to make use of requests library).
    
Extract the zip file, again programmatically, similar to how we did it in class.

You can download the data and unzip it manually. But we learned about APIs, so we can do it in a reproducible way with the requests library, similar to how we did it in class.

There are 5 files in the figshare repo. The one we want is: `data.zip`
</div>

### 3.1 Setting up API

Code adopted from lecture 2 notes.

<!--  -->

In [2]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"

In [3]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

### 3.2 Download the data and unzip it

Code adopted from lecture 2 notes.

In [4]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: total: 3.28 s
Wall time: 25.5 s


In [5]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: total: 21.6 s
Wall time: 22.1 s


In [6]:
%ls figsharerainfall/

Invalid switch - "".


### 3.3 Preview data

In [7]:
pd.read_csv("figsharerainfall/ACCESS-CM2_daily_rainfall_NSW.csv")

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-36.25,-35.00,140.625,142.50,3.293256e-13
1,1889-01-02 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
2,1889-01-03 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
3,1889-01-04 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
4,1889-01-05 12:00:00,-36.25,-35.00,140.625,142.50,1.047658e-02
...,...,...,...,...,...,...
1932835,2014-12-27 12:00:00,-30.00,-28.75,151.875,153.75,2.951144e-02
1932836,2014-12-28 12:00:00,-30.00,-28.75,151.875,153.75,2.257118e-01
1932837,2014-12-29 12:00:00,-30.00,-28.75,151.875,153.75,1.204670e-01
1932838,2014-12-30 12:00:00,-30.00,-28.75,151.875,153.75,2.632404e-02


In [8]:
%load_ext memory_profiler

<!--  -->

<!--  -->

## 4. Combining data CSVs

rubric={correctness:10,reasoning:10}

<div class="alert alert-block alert-info">

1. Combine data CSVs into a single CSV using pandas.

2. When combining the CSV files, add an extra column called "model" that identifies the model. 
    > Tip 1: you can get this column populated from the file name, eg: for file name "SAM0-UNICON_daily_rainfall_NSW.csv", the model name is SAM0-UNICON 
    
    > Tip 2: Remember how we added year when we combined airline CSVs. Tip 3: You can use regex generator.

Note: There is a file called observed_daily_rainfall_SYD.csv in the data folder that you downloaded. Make sure you exclude this file (programmatically or just take out that file from folder) before you combine CSVs. We will use this file in our next milestone.
</div>

- As instructed, the code below removes the `observed_daily_rainfall_SYD.csv` file 

In [9]:
# Removing the observed_daily_rainfall_SYD.csv file as instructed

file_path = 'figsharerainfall/observed_daily_rainfall_SYD.csv'

try:
    os.remove(file_path)
except OSError as e:
    print("Error: %s : %s" % (file_path, e.strerror))

- Note that a new column, `model` that identify the respective files is created below

In [10]:
%%time
%%memit

files = glob.glob('figsharerainfall/*.csv')
df = pd.concat(
    (pd.read_csv(file, index_col=0)
                .assign(model=re.findall(r'(?<=fall\\)(.+)?(?=_daily)', file)[0])
                for file in files)
              )
df.to_csv("figsharerainfall/combined_data.csv")

peak memory: 7039.04 MiB, increment: 6765.08 MiB
CPU times: total: 6min 36s
Wall time: 6min 43s


In [11]:
print(df.shape)

df.head()

(62467843, 6)


,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


The above code takes a very long to run. We will be trying in-memory methods below:
 - Float convert 
 - Chunks
 - Dask

<!--  -->

<!--  -->

## 5. Load the combined CSV to memory and perform a simple EDA

rubric={correctness:10,reasoning:10}

<div class="alert alert-block alert-info">

Investigate at least two of the following approaches to reduce memory usage while performing the EDA (e.g., value_counts).
- Changing dtype of your data
- Load just columns what we want
- Loading in chunks
- Dask

Compare run times on different machines within your team and summarize your observations.
</div>

- Note here that our EDA of choice is simply returning the count of each unique model using `value_counts`

<!--  -->

### Method 1: Changing dtype (from float64 to float32)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62467843 entries, 1889-01-01 12:00:00 to 2014-12-31 12:00:00
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   lat_min        float64
 1   lat_max        float64
 2   lon_min        float64
 3   lon_max        float64
 4   rain (mm/day)  float64
 5   model          object 
dtypes: float64(5), object(1)
memory usage: 3.3+ GB


#### float64

In [13]:
%%time
%%memit

# Default float64 dtypes
df = pd.read_csv("figsharerainfall/combined_data.csv", index_col=0)
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
peak memory: 9930.57 MiB, increment: 6218.98 MiB
CPU times: total: 1min 10s
Wall time: 1min 11s


#### float32

- Here, we are changing the data types of all `float64` datatype to `float32` datatype. This is so we can see if we have improved speed as a result

In [14]:
%%time
%%memit

# Changing dtype from float64 to float32
df_32 = pd.read_csv("figsharerainfall/combined_data.csv", index_col=0).loc[:, df.columns != "model"].astype('float32')
df_32["model"] = df["model"]
print(df_32["model"].value_counts())

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
peak memory: 13285.80 MiB, increment: 6217.06 MiB
CPU times: total: 1min 20s
Wall time: 1min 23s


In [15]:
df_32.info() # Double check data type

<class 'pandas.core.frame.DataFrame'>
Index: 62467843 entries, 1889-01-01 12:00:00 to 2014-12-31 12:00:00
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   lat_min        float32
 1   lat_max        float32
 2   lon_min        float32
 3   lon_max        float32
 4   rain (mm/day)  float32
 5   model          object 
dtypes: float32(5), object(1)
memory usage: 2.1+ GB


- As can be seen above, there is a slight decrease in run time with `float32`.

<!--  -->

### Method 2: Loading in Chunks

In [16]:
%%time
%%memit

counts = pd.Series(dtype=int)
for chunk in pd.read_csv("figsharerainfall/combined_data.csv", chunksize=10_000_000, index_col=0, parse_dates=True):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int32
peak memory: 11083.07 MiB, increment: 1862.54 MiB
CPU times: total: 1min 45s
Wall time: 1min 47s


- Loading the data in chunks has resulted in even less run time and memory for most of the team with the exception of people with high RAM.

<!--  -->

### Method 3: Dask

In [17]:
%%time
%%memit

dash_df = dd.read_csv("figsharerainfall/combined_data.csv", parse_dates=["time"])

peak memory: 9362.81 MiB, increment: 7.25 MiB
CPU times: total: 109 ms
Wall time: 3.44 s


In [18]:
%%time
%%memit

print(dash_df["model"].value_counts().compute())

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
peak memory: 10938.25 MiB, increment: 1582.95 MiB
CPU times: total: 2min 5s
Wall time: 1min 13s


- `Dask` has proven to be the fastest in-memory method for M1, beating both `chunking` and `changing dtype` methods. But for the rest of the team it was the slowest.

<!--  -->

<!--  -->

## Comparing Observations

- The table below summarizes machine specifications and speed observations of members of group 19

| Team Member    | Operating System | RAM   | Processor                       | Is SSD | Time taken (Combining data) | Time taken (float64) | Time taken (float32) | Time taken (Chunks) | Time taken (Dask) |
|----------------|------------------|-------|---------------------------------|--------|-----------------------------|----------------------|----------------------|---------------------|-------------------|
| Victor Francis | MacOS Monterey   | 16 GB | Apple M1 Pro                    | Yes    | 6min 53s                    | 1min 3s              | 57.9 s               | 52.6 s              | 823 ms            |
| John Lee       | Windows 10 Pro   | 16 GB | Intel(R) Core(TM) i7            | Yes    | 9min 2s                     | 1min 51s             | 1min 46s             | 1min 37s            | 2min 10s          |
| Wanying Ye     | MacOS Monterey   | 8 GB  | 2.9 GHz Dual-Core Intel Core i5 | Yes    | 20min 26s                   | 4min 40s             | 3min 45s             | 2min 49s            | 2min 5s           |
| Katia Aristova |Windows 10 Pro|32 GB|11th Gen Intel(R) Core(TM) i7-1165G7|Yes|5min 45s|1min 5s|1min 2s|1min 34s|1min 56s|

<!--  -->

<!--  -->

<!--  -->

## 6. Perform a simple EDA in R
rubric={correctness:15,reasoning:10}


<div class="alert alert-block alert-info">

Pick an approach to transfer the dataframe from python to R.
 
- Parquet file
- Feather file (explored in the Appendix)
- Pandas exchange
- Arrow exchange (we picked this option)

Discuss why you chose this approach over others.

</div>

<!--  -->

In [19]:
%load_ext rpy2.ipython

C:\Users\kater\miniconda3\envs\525_2022\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [20]:
import pyarrow.dataset as ds
import pyarrow as pa
import pandas as pd
import pyarrow 
from pyarrow import csv
import rpy2_arrow.pyarrow_rarrow as pyra

### Chosen Method: Using Apache Arrow

In [21]:
%%time
%%memit
dataset = ds.dataset("figsharerainfall/combined_data.csv", format="csv")
table = dataset.to_table()
r_table = pyra.converter.py2rpy(table)

peak memory: 13559.82 MiB, increment: 4032.96 MiB
CPU times: total: 1min 18s
Wall time: 1min 17s


In [22]:
%%time
%%R -i r_table

start_time <- Sys.time()
suppressMessages(library(dplyr))
result <- r_table %>% count(model)
end_time <- Sys.time()
print(result %>% collect())
print(end_time - start_time)

# A tibble: 27 x 2
   model                  n
   <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# ... with 17 more rows
Time difference of 0.440733 secs
CPU times: total: 2.56 s
Wall time: 2.61 s


### Discuss why you chose this approach over others

`Apache Arrow` was chosen because of its superior memory storage capability. It utilizes in-memory data methodology, hence leading to improved speed and computational efficiency. A known fact is that regular serialization/deserialization (`Pandas Exchange`) technique requires a significant amount of time and memory. Apache Arrow solves this problem. As for the computing time, `Arrow` takes 1m 7s to convert and then 1.58 s to calculate, this makes it faster than EDA processes in pandas.

Also, `Feather` would have been a worthy alternative, but it's still in developmental stage and as such, has some shortcomings. The benefits of feather is its optimization for working with both pandas and R dataframes, as well as the fact that it is language-agnostic. The downside is that Feather is not suitable for long-term storage. Feather takes about 1.5 minutes to convert from pandas to feather format. However, once done, it is very fast and can work with a variety of languages (~3.2 seconds EDA). In a scenario where the dataset would not be stored long term, it is the most elegant solution (see Appendix below for a `Feather` demonstration).

`Parquet` format for the most part is used for archival storage purposes, which is not our intent here. Therefore, Parquet format isn't the best solution in this case.

Conclusively, computational efficiency, in-memory data capability, ephemeral storage suitability are why we have gone with the choice of Apache Arrow.

## Appendix

### Feather method

In [4]:
%%time
df_2 = pd.read_csv("figsharerainfall/combined_data.csv")
df_2.to_feather("figsharerainfall/combined_data_f.feather")
dfeather = pd.read_feather("figsharerainfall/combined_data_f.feather")

CPU times: total: 1min 20s
Wall time: 1min 12s


In [1]:
%load_ext rpy2.ipython

C:\Users\kater\miniconda3\envs\525_2022\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [2]:
%%R

suppressMessages(library(dplyr))
suppressMessages(library(arrow))

start_time <- Sys.time()
feather_r <- read_feather("figsharerainfall/combined_data_f.feather")
result <- feather_r  %>% count(model)
print(result)
end_time <- Sys.time()
print(end_time - start_time)

# A tibble: 27 x 2
   model                  n
   <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# ... with 17 more rows
Time difference of 7.050824 secs
